In [1]:
# StyleGAN2の学習を行うサンプルコード

In [2]:
# GPU情報の確認
!nvidia-smi

Sun Feb  7 12:15:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# google driveからcolaboにファイルをコピーするメソッド
import requests
from bs4 import BeautifulSoup

def download_file_from_google_drive(id, destination):
    print('---- download_file_from_google_drive start ----')
    URL = "https://docs.google.com/uc?export=download"

    try:
        session = requests.Session()

        response = session.get(URL, params = { 'id' : id }, stream = True)
        print('response.status_code: {}'.format(response.status_code))
        print('response.headers: {}'.format(response.headers))

        soup = BeautifulSoup(response.text, 'html.parser')
        print('===== soup ====')
        print(soup)

        id, cofirm_token = get_confirm_token(soup)
        print(f'id: {id}')
        print(f'confirm_token: {cofirm_token}')

        params = { 'id' : id, 'confirm' : cofirm_token }
        # params = { 'id' : id }
        
        response = session.get(URL, params = params, stream = True)

        save_response_content(response, destination)
    except Exception as e:
        print(e)


def get_confirm_token(soup):
    # サイズの大きいファイルをダウンロードしようとすると、「ファイルをプレビューできませんでした」というメッセージ画面が表示される
    # このメッセージ画面のhtmlからダウンロードボタンをクリックした際のidとconfirmを取得する
    # このメッセージ画面はGoogleの仕様によって変更されて動作しなくなる場合があります。
    # その際はid等の情報を取得するようにコードを変えるか、
    # https://drive.google.com/file/d/1LM4FtUltzS45PuFyfuSp3I8QdTD8Cu0F/view?usp=sharing
    # 上記のURLから手動でファイルを取得し、content/dataset配下に配置してください

    print(' **** get_confirm_token ****')
    form = soup.find('form', {'id': 'download-form'})
    print(form)

    # フォームアクションURLからIDとconfirmトークンを抽出
    action_url = form.get('action')
    params = action_url.split('&')
    file_id = params[1].split('=')[1]
    confirm_token = params[2].split('=')[1]

    print('file_id: {}'.format(file_id))
    print('confirm_token: {}'.format(confirm_token))
    
    return file_id, confirm_token


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)



In [ ]:
# 実行環境のrootディレクトリの確認
%cd /content/
!ls -lah

In [ ]:
# ソースコードのダウンロード
%cd /content/
!rm -fr gan_sample
!git clone https://github.com/ayukat1016/gan_sample.git

In [6]:
# datasetを配置するディレクトリを作成します。
!mkdir ./dataset

In [ ]:
# endless_summer datasetのダウンロードを行います。
# ★★★★ 7GBあるのでダウンロードに時間がかかります。 ★★★★
# https://drive.google.com/file/d/1LM4FtUltzS45PuFyfuSp3I8QdTD8Cu0F/view?usp=sharing
file_id = '1LM4FtUltzS45PuFyfuSp3I8QdTD8Cu0F'
destination = './dataset/endless_summer.zip'
download_file_from_google_drive(file_id, destination)

In [ ]:
# ダウンロードしたendless summer datasetを展開します
!unzip ./dataset/endless_summer.zip -d ./dataset

In [ ]:
# データ件数の確認
!find dataset/* -type f | wc -l

In [ ]:
# Google driveのマウントを行います。
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# マウントしたGoogle Drive情報の確認
!ls gdrive -lah

In [ ]:
# Tensorboardの起動
# 学習経過を確認するためにTensorboardを使用します
# --log_dirオプションに各自のGoogle Driveのディレクトリを指定してください。

%cd /content/
%load_ext tensorboard
%tensorboard --logdir=./gdrive/MyDrive/'Colab Notebooks'/gan_sample/chapter7/logs

In [ ]:
# 学習を行います。
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# ★★★★★　長時間の学習を行うとGoogle Driveの容量が足りなくなる事があるので、注意してください!!!!!!★★★★
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# Google Driveの容量が少ない方は事前に容量を購入するか、下記のsave intervalの値を大きくしてください。
# --save_model_interval=32 \
# --save_metrics_interval=2 \
# --save_images_tensorboard_interval=8 \
# --save_images_interval=32 \
# 特に--model_pathに保存されるモデルファイルは1ファイルで300MBと大きいので、古いモデルファイルは消すようにしてください。

# それっぽい画像が生成されるまで、2〜3日、精度の良い画像が生成されるには1週間以上かかるかもしれません。
# 学習は--model_pathオプションに格納されている一番新しいモデルファイルを使用して継続学習を行います。
# --model_pathで指定したディレクトリにモデルファイルがない場合ははじめから学習を行います。
# fidスコア算出のためにinceptionの重みファイルを初回にダウンロードするので、学習が開始するまで時間(5分〜10分)がかかります。

# --model_pathに学習したモデルの保存先を指定します。
# --data_pathに学習するdata setのディレクトリを指定します。
# --resultsに学習した結果を保存するディレクトリを指定します。
# --cache_pathに一時ファイルの格納先を指定します。
# --tensorboad_pathにTensoboardのログファイルを保存するディレクトリを指定します。
# --save_model_interval: モデルを保存する間隔を指定します。32を指定すると32回のループ回数毎にモデルを保存します。
# --save_metrics_interval: 指標をTensorboardに出力する間隔を指定します。
# --save_images_tensorboard_interval: 生成した画像をTensorboadに出力する間隔を指定します。
# --save_images_interval: 生成した画像をファイルとして保存する間隔を指定します。
# --generator_train_num: generatorが連続して学習する回数を指定します。とりあえず4で良いと思います。
# --discriminator_train_num: discriminatorが連続して学習する回数を指定します。とりあえず4で良いと思います。
# --reverse_decay: 1のままで
# --g_reg_interval: generatorの正則処理を行う間隔を指定します。とりあえず4で良いと思います。
# --d_reg_interval: discriminatorの正則処理を行う間隔を指定します。とりあえず16で良いと思います。
# --fid_score_interval: fidスコアを算出する間隔をしてします。fidのスコアの算出は非常に時間がかかるので、実行したくない場合は大きな値を指定してください。

%cd /content/gan_sample/chapter7/stylegan2_pytorch

!python training.py --batch_size=4 --resolution=512 \
--model_path=/content/gdrive/MyDrive/'Colab Notebooks'/gan_sample/chapter7/model \
--data_path=../../../dataset/endless_summer \
--results=/content/gdrive/MyDrive/'Colab Notebooks'/gan_sample/chapter7/results \
--cache_path=/content/gdrive/MyDrive/'Colab Notebooks'/gan_sample/chapter7/cache \
--tensorboard_path=/content/gdrive/MyDrive/'Colab Notebooks'/gan_sample/chapter7/logs \
--save_model_interval=32 \
--save_metrics_interval=2 \
--save_images_tensorboard_interval=8 \
--save_images_interval=32 \
--generator_train_num=4 \
--discriminator_train_num=4 \
--reverse_decay=1 \
--g_reg_interval=4 \
--d_reg_interval=16 \
--fid_score_interval=2048